# Explainer notebook - Songwriter, artists and song project

------------------------------------

# Motivation

#### What is your dataset?
Our dataset contains the top 50 songs from the top artists of the decade of 2010. Based on the top 100 list of artists from pulse music, we fetched the top 50 songs for each of the 100 artists. We have then expanded the dataset by adding select features such as all featuring artists of the songs, the
release year, the duration of the song, popularity score from Spotify, all songwriters listed on the song and lastly each song’s lyrics. We constructed the dataset ourself by using multiple different API’s - specifically the Genius API, Spotify API and Musicbrainz API.½

#### Why did you choose this/these particular datasets?

We wanted to research what the connection between present day artists or songs was by looking at how the artists and songs might be connected by common songwriters or lyrics. We wanted to expand this research by both looking at what connection this might have to the popularity of the song and how the themes in the lyrics might be influenced or depend on the songwriters and how these themses vary and change depending on the release year. 

We decided to use a list of the top 100 artists from pulse music in order to narrow down the artists to look at. Here our thought process was that the artists with the most means and oppertunities would be the ones the most popular and the ones appearing in the top 50. These artists would have the most ressources to bring in the best songwriters which could possibly create a network behind the scenes of the music industry with these songwriters being the mastermind of the most famous artists. 

Through Genius we could fetch lyrics for all the songs, from musicBrainz we got all the songwriters and from spotify we could fetch the remaining attributes we have included.

#### What was your goal for the end user's experience?

We had both heard a bit here and there on the internet and through other people that a lot of popular artists do not write their own songs and that many popular songs share common songwriters. It made us curious to actually map out how music industry was structured behind the scenes of the most popular artists that we are presented with on the stage and in the radio. The main goal has been to give the end user an engaging and insightful exploration of the music landscape by uncovering the hidden connections between artists, songwriters and songs. To achieve this we have focused in on 2 different areas:

1. Highlighting songwriters, how they are used by different artists and the difference here between top artists.
2. Determining popular themes and trends in songs and whether they depend on the songwriter


 On the surface it might not be clear which artists are actual central nodes and which artist or songwriters have the most influence. Here, we try to highlight the most influential songwriters who play a significant role behind the scenes. We aimed to visualize how some artists are also interlinked through shared songwriters, enabling users to discover collaborative patterns and creative networks. We aimed to identify and map potential communities within the artist and song networks connected by songwriters and thereby offering users a broader understanding of the structure and dynamics of the music industry. By including popularity we also had a way of defining succes in these communities. The dataset and network enabled us to compare of how the different top artists utilize songwriters, possibly revealing strategic or stylistic differences.

For the second goal we include the lyrics of the songs in order for us to do semantic analysis to uncover thematic patterns to add to the end user's experience. By analyzing lyrics based on their release year and popularity levels, we aimed to reveal how themes evolve and what actually resonates with the audience — such as shifting from love and heartbreak to empowerment or social commentary. This allows users to explore how artists and songwriters reflect cultural changes through language. With the lyrics we provide insights into what recurring motifs, emotional tone, and general language trends are present in the lyrics. Ultimately, the goal was to let users interact with the lyrical content on a deeper level. 

---------------------------------

# Basic Stats

Packages

In [23]:
#GENERAL
import pandas as pd
import requests
import time
from tqdm import tqdm
from ratelimit import limits, sleep_and_retry
from bs4 import BeautifulSoup
import unicodedata
import builtins
import ast

#SPOTIFY
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException

#MUSIC BRAINZ
import musicbrainzngs

#GENIUS
import lyricsgenius
import re

## Constructing dataset

When we constructed the dataset we did through multiple rounds. First we fetched the top 50 songs for each of the top 100 artists but to no risk loosing data if the process for some reason was interrupted we split the 100 artists into 2 and after each round we saved it as a csv. We then for each of these csv files first fecthed all lyrics and saved those as seperate dataframes and also for the songwriters, which were also saved as seperate csv files. At the end when we had collected all the data for all songs, we combined the multiple csv files into one dataset which is saved as final_dataset.csv. 

##### Fetch top 50 songs from top 100 artists with Spotify API

In [ ]:
# Safe retry function
def safe_spotify_call(call_func, *args, sleep=0.4, **kwargs):
    time.sleep(sleep)
    
    try:
        return call_func(*args, **kwargs)
    
    #Depending on the type of spotify API error retry
    except SpotifyException as e:
        # Error "Too Many Requests"
        if e.http_status == 429:
            retry_after = int(e.headers.get("Retry-After", 10))
            print(f"Rate limit hit. Sleeping for {retry_after} seconds...")
            time.sleep(retry_after + 1)
            return safe_spotify_call(call_func, *args, **kwargs)
        else:
            print(f"Spotify error: {e}")
            return None

    #network error
    except requests.exceptions.RequestException as e:
        print(f" Network error: {e}")
        time.sleep(5)
        return safe_spotify_call(call_func, *args, **kwargs)


In [ ]:
def get_top_50_songs_by_artist(artist_query, market="US"):
    #SPOTIFY API credentials 
    SPOTIFY_CLIENT_ID = "449143cf15fe468e94f3c16d7a51c17f"
    SPOTIFY_CLIENT_SECRET = "60259a57c2f14c5190a68381fa4bb013"
    
    #Use credentials
    auth_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
    spotify = spotipy.Spotify(auth_manager=auth_manager)

    #Get artist based on ID
    if len(artist_query) == 22 and " " not in artist_query:
        artist_id = artist_query
        artist_info = safe_spotify_call(spotify.artist, artist_id)
        artist_name = artist_info["name"]

    #Get artist based on name
    else:
        result = safe_spotify_call(spotify.search, q=f"artist:{artist_query}", type="artist", limit=5)

        #No artist match
        if not result['artists']['items']:
            print(f"No artist found for query: {artist_query}")
            return pd.DataFrame()
        
        #Select most popular match
        best_match = max(result['artists']['items'], key=lambda x: x['popularity'])
        artist_id = best_match['id']
        artist_name = best_match['name']

    #Get all albums from artist
    album_result = safe_spotify_call(spotify.artist_albums, artist_id, album_type='album,single,compilation', limit=50)
    albums = album_result.get('items', [])
    album_ids = list({album['id'] for album in albums})

    #Step 3 :Get all tracks from artist
    track_info = {}
    for album_id in album_ids:
        try:
            album = safe_spotify_call(spotify.album, album_id)
            if not album:
                continue
            release_year = album.get("release_date", "1900").split("-")[0]
            tracks = safe_spotify_call(spotify.album_tracks, album_id)
            if not tracks:
                continue
            for track in tracks['items']:
                track_id = track['id']
                track_name = track['name']
                track_artists = [a['name'] for a in track['artists']]
                artist_names = ", ".join(track_artists)

                if artist_name.lower() not in [a.lower() for a in track_artists]:
                    continue

                if track_name not in track_info:
                    track_info[track_name] = {
                        'id': track_id,
                        'name': track_name,
                        'artists': artist_names,
                        'release_year': release_year
                    }
        time.sleep(0.1)

    #Add track popularity and duration
    tracks = list(track_info.values())
    for i in range(0, len(tracks), 50):
        batch = tracks[i:i + 50]
        ids = [t['id'] for t in batch]
        full_tracks = safe_spotify_call(spotify.tracks, ids)
        if full_tracks:
            for t, ft in zip(batch, full_tracks['tracks']):
                t['popularity'] = ft['popularity']
                t['duration_ms'] = ft['duration_ms']
        time.sleep(0.2) 

    #Return top 50 by popularity
    df = pd.DataFrame(tracks)
    if df.empty:
        return df
    df = df.sort_values(by="popularity", ascending=False).head(50)
    return df[['name', 'artists', 'popularity', 'duration_ms', 'release_year']]

In [ ]:
#List of artists
top_50_artists_2010 = [
    "Drake", "Taylor Swift", "Bruno Mars", "Rihanna", "4dpARuHxo51G3z768sgnrY",
    "Ed Sheeran", "Justin Bieber", "Katy Perry", "Maroon 5",
    "Post Malone", "Lady Gaga", "Ariana Grande", "Imagine Dragons", "Nicki Minaj","Eminem","Luke Bryan","Pink","One Direction","Justin Timberlake",
    "Kendrick Lamar","Lady Antebellum","Beyonce","Jason Aldean","Sam Smith","Kesha","Flordia Georgia Line","Twenty one pilots","Lil Wayne","Chris Brown",
    "Blake Shelton","Travis Scott","Khalid","Shawn Mendes","Cardi B","Future","Mumford & sons","Selena Gomez","Jay-Z","Meghan Trainor","J. Cole","Usher",
    "Coldplay","The Black Eyed Peas","Pitbull","Flo Rida","Michael Buble","Zac Brown Band","Jason Derulo"
]

top_50_100_artists_2010 = [
    "The chainsmokers","Halsey","Lorde","Kanye West","Kenny Chesney","Miley cyrus","Carrie Underwood","Wiz Khalifa","Migos","Kelly Clarkson","OneRepublic",
    "Macklemore & Ryan Lewis","XXXtentacion","Eric Church","Juice Wrld","fun.","Billie Ellish","LMFAO","DJ khaled","Chris stapleton","Calvin harris","Britney spears",
    "Fetty wap","Sia","Pentatonix","David Guetta","U2","Ellie Goulding", "The lumineers","Pharrell Williams","The rolling stones","Train","Trey songz",
    "Demi lovato","Sam hunt","Big sean","Camila cabello","lil uzi vert","Panic! at the disco","Miranda lambert","Bruce springsteen","John legend","B.o.B",
    "21 savage","Thomas Rhett","Meek mill","Keith Urban","Bon jovi","5 seconds of summer","paul mccartney"
]

# Empty dictionary to store results
artist_song_dfs = {}

# Loop over each artist and fetch top 50 songs
for artist in top_50_artists_2010:
    df = get_top_50_songs_by_artist(artist)
    artist_song_dfs[artist] = df

In [ ]:
# Merge all artist dataFrames
all_songs_df = pd.concat(artist_song_dfs.values(), keys=artist_song_dfs.keys(), names=["Artist"])

#Reset index
all_songs_df = all_songs_df.reset_index(level="Artist").reset_index(drop=True)

#Drop NaN
all_songs_df.dropna()

In [ ]:
all_songs_df.to_csv('first_50_artists.csv', index=False)

#### Fetch songwriters for each song with MusicBrainz API

We saved the dataframe from the previous fetching as a cvs to ensure we did not loose data. We then loaded this in to add the songwriters for each song. 

In [ ]:
# Rate limit - 1 call per second
CALLS = 1
PERIOD = 1 

@sleep_and_retry
@limits(calls=CALLS, period=PERIOD)

#Safe function to search works
def safe_search_works(title, artist):
    return musicbrainzngs.search_works(query=title, artist=artist, limit=1)

@sleep_and_retry
@limits(calls=CALLS, period=PERIOD)

#Safe function to get work
def safe_get_work_by_id(work_id):
    return musicbrainzngs.get_work_by_id(work_id, includes=["artist-rels"])

#Safe function to get songwriters
def get_songwriters_from_musicbrainz(title, artist):
    writers = []

    try:
        work_result = safe_search_works(title, artist)
        if work_result['work-list']:
            work_id = work_result['work-list'][0]['id']
            work_data = safe_get_work_by_id(work_id)
            for rel in work_data['work'].get("artist-relation-list", []):
                if rel['type'] in ["composer", "lyricist", "writer"]:
                    writers.append(rel['artist']['name'])
    except Exception as e:
        print(f"[MusicBrainz Writers Error] {title} - {artist}: {e}")

    return ", ".join(writers)

In [ ]:
#Load dataset with songs from SPOTIFY fetch
all_songs = pd.read_csv("first_50_artists.csv")

In [ ]:
#Break all_songs dataframe into smaller batches
all_songs = all_songs[:1000]
all_songs

In [ ]:
#Fecth songwriters for all songs
tqdm.pandas()
all_songs["songwriters"] = all_songs.progress_apply(lambda row: get_songwriters_from_musicbrainz(row["name"], row["Artist"]), axis=1)

In [ ]:
all_songs.to_csv('first_50_artists_with_songwriters.csv', index=False)

#### Fetch lyrics for each song with GENIUS API

Again here, we saved the dataframe from the previous fetching as a cvs to ensure we did not loose data. We then loaded this in to add the lyrics for each song. 

In [ ]:
# Genius API setup
genius = lyricsgenius.Genius("ZVjXsUV0Xb_wHopoibicTBRVRv8eJvdu4i7NL5mW2hcITfM6W2cvk4ha6kBktTNL", timeout=15)
genius.skip_non_songs = True
genius.excluded_terms = ["(Remix)", "(Live)"]

# Common language names to filter
LANGUAGE_NAMES = {
    "English", "Español", "Français", "Deutsch", "Português", "Русский", "العربية", "日本語", "Türkçe",
    "한국어", "Polski", "Italiano", "Українська", "Tiếng Việt", "Bahasa Indonesia", "Svenska", "Norsk",
    "Ελληνικά", "עברית", "Magyar", "Čeština", "Slovenčina", "Беларуская", "Català", "Македонски", "Српски",
    "ไทย", "Česky", "Română", "हिन्दी"
}

def is_probably_language_line(line):
    return (
        line in LANGUAGE_NAMES or
        (line.isalpha() and line.istitle() and len(line.split()) == 1)
    )

def is_non_latin_text(line):
    return all(
        not unicodedata.name(char, "").startswith("LATIN") 
        for char in line if char.isalpha()
    )

def get_lyrics(song_title, artist_name):
    try:
        original_print = builtins.print
        builtins.print = lambda *args, **kwargs: None

        song = genius.search_song(song_title, artist_name)

        builtins.print = original_print
        if not song:
            return None

        page = requests.get(song.url)
        soup = BeautifulSoup(page.text, "html.parser")

        lyrics_divs = soup.select("div[data-lyrics-container='true']")
        lyrics_lines = []

        for div in lyrics_divs:
            for line in div.stripped_strings:
                line = line.strip()

                if (
                    line.startswith("[") and line.endswith("]") or
                    is_probably_language_line(line) or
                    is_non_latin_text(line) and len(line) < 20 or
                    "translation" in line.lower() or
                    "contributors" in line.lower() or
                    "edit" in line.lower() or
                    line.lower().endswith("lyrics") or
                    "read more" in line.lower() or
                    line.lower().startswith("“") or
                    "is a track that" in line.lower() or
                    "/" in line or "(" in line or ")" in line or
                    line.lower().startswith("concluding ") or
                    line.lower().startswith("this track") or
                    line.lower().startswith("the song") or
                    len(line) < 10 and not any(char.isalpha() for char in line) or
                    line.endswith("…") or "…" in line
                ):
                    continue

                lyrics_lines.append(line)

        return "\n".join(lyrics_lines).strip() if lyrics_lines else None

    except Exception:
        return None


In [ ]:
#Remove anything containing feature in the title
def remove_feat(text):
    return re.sub(r"[\(\[\-]?\s*(feat\.?|ft\.?)\s+[^\)\]]+[\)\]]?", "", text, flags=re.IGNORECASE).strip()

all_songs["name"] = all_songs["name"].apply(remove_feat)


In [ ]:
songs_to_scrape = pd.read_csv("first_50_100_artists.csv")

In [ ]:
tqdm.pandas()

def scrape_with_delay(row):
    lyrics = get_lyrics(row["name"], row["Artist"])
    time.sleep(2)  # Safe delay to prevent rate-limiting
    return lyrics

songs_to_scrape["lyrics"] = songs_to_scrape.progress_apply(scrape_with_delay, axis=1)

In [ ]:
songs_to_scrape.to_csv('first_50_artists_with_lyrics.csv', index=False)


## Data cleaning and preprocessing

In each of the 3 fetching steps we included datacleaning as a way of ensuring the data was clean for the next step. For all 3 steps we removed any rows containing NaN as we only wanted songs which had all features. 

**SPOTIFY API** $\newline$
When fetching the top 50 songs for the top artists we experienced that some artists have shared artistic names, which meant that we were retrieving wrong information. To overcome this we looked up the specific ID for the artists causing problems and used these instead to ensure we fetched the correct artist. 

**GENIUS** $\newline$
The GENIUS API had multiple extra information besides the lyrics when we fetched it such as including languages and descriptive lines for the content. To overcome this we introduced a list of common languages where lines containing these should be removed. Similarly the descriptive lines contained specific enough words that we were able to remove them and single out just the lyrics to add to the dataset. 

**SOURCES OF ERROR:** $\newline$
We discovered during our work with the dataset that some artists names where replaced by other artists such as pink vs pink floyd and aditionally that some artists even though they appeared in the dataset were skipped in the code such as The Black Eyed Peas, U2 and fun. We also realised that we had mistakenly missed two artists "The Weeknd" and "Florida Georgia Line", which was a mistake we did not catch when fetching the dataset. Lastly we also ran had changed some Adele's name to an ID code, but forgot to change it back afterwards, which also lead to her not being included. Due to time restrainst at this point, we could not fetch the data again, and therefore decided to remove the problematic artists and continue without the missing artists. Furthermore in the top 100 list there was a group based around covers for kids, which we also removed from our list of top artists. This is the explanation for the missing artists in the following data stats. 

##### Basic stats for the dataset: 

**SONGS:** 3731 $\newline$
**SONGWRITERS:** 4498 $\newline$
**ARTISTS:** 91 $\newline$


### LYRICS TOKENIZING/PREPROCESSING - CLARA!!!!!

## Dataset stats 

This ended up being our final dataset. Artist is the main artist, name is the song name, artists are the featuring artists, popularity is a score from SPOTIFY and songwriters contains both componist, lyricist and writer: 


In [25]:
df = pd.read_csv("final_dataset.csv")
df

,Artist,name,artists,popularity,duration_ms,release_year,songwriters,lyrics
0,the chainsmokers,Something Just Like This,"['the chainsmokers', 'coldplay']",90.0,247160.0,2017,"['jonny buckland', 'will champion', 'chris mar...","I've been reading books of old, the legends an..."
1,the chainsmokers,Paris,['the chainsmokers'],82.0,221506.0,2017,['elijah the alchemist'],We were stayin' in Paris\nTo get away from you...
2,the chainsmokers,This Feeling,"['the chainsmokers', 'kelsea ballerini']",74.0,197946.0,2018,"['travis barker', 'tom delonge', 'mark hoppus']",I'll tell you a story before it tells itself\n...
3,the chainsmokers,Jungle,"['alok', 'the chainsmokers', 'mae stephens']",74.0,175500.0,2023,"['the motels', 'the motels']","I'm out, feeling lonely again\nAm I too scared..."
4,the chainsmokers,Who Do You Love,"['the chainsmokers', '5 seconds of summer']",70.0,226733.0,2019,['the escape club'],Found cigarettes in your Fendi coat\nEven thou...
...,...,...,...,...,...,...,...,...
3726,kendrick lamar,Purple Hearts,"['kendrick lamar', 'summer walker', 'ghostface...",67.0,329295.0,2022,"['westside boogie', 'sam dew', 'ghostface kill...","Mmm, tippy-toeing and I'm mud-walkin', hmm\nRo..."
3727,kendrick lamar,Swimming Pools (Drank) - Extended Version,['kendrick lamar'],67.0,313786.0,2012,"['kendrick lamar', 'nikhil seetharam', 't‐minus']","Now, I done grew up round some people\nLivin' ..."
3728,kendrick lamar,How Much A Dollar Cost,"['kendrick lamar', 'james fauntleroy', 'ronald...",66.0,261746.0,2015,"['james fauntleroy', 'ronald isley', 'kendrick...",How much a dollar really cost?\nThe question i...
3729,kendrick lamar,Hol' Up,['kendrick lamar'],66.0,173158.0,2011,"['kendrick lamar', 'sounwave']",I wrote this record while thirty thousand feet...


In [26]:
def safe_parse(value):
    try:
        parsed = ast.literal_eval(value)
        return parsed if isinstance(parsed, list) else [parsed]
    except:
        return [value]


df['songwriters'] = df['songwriters'].apply(safe_parse)
df = df.explode('songwriters').reset_index(drop=True)

In [27]:
df["Artist"].value_counts()

Artist
coldplay              284
travis scott          282
meghan trainor        263
beyonce               261
kendrick lamar        260
                     ... 
b.o.b                  70
the rolling stones     58
bruce springsteen      51
paul mccartney         28
chris stapleton        20
Name: count, Length: 91, dtype: int64

In [28]:
df["songwriters"].value_counts()

songwriters
max martin         106
meghan trainor      72
benny blanco        70
jay‐z               68
louis bell          66
                  ... 
siobhán donaghy      1
felix howard         1
john lipsey          1
cameron mcvey        1
rob moose            1
Name: count, Length: 4564, dtype: int64

----------------------------

# Tools, theory and analysis

##### Describe the process of theory to insight

#### How did you use the tools to understand your dataset?

## Dataset analysis

## Networks creation and analysis

#### Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.

## Semantic analysis - Lyrics

#### Talk about how you've worked with text, including regular expressions, unicode, etc.

--------------------------------

# Discussion

#### What went well?


#### What is still missing? What could be improved? Why?